In [ ]:
# !pip install textblob

# !pip install tweepy

# !pip install pycountry

# !pip install wordcloud 

# !pip install langdetect

In [1]:
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
from datetime import timedelta
import datetime
import pytz

In [ ]:
# import config
# # put ur keys in a py file as a dictionary and import them so u can keep em secret
# consumer_key = config.params['consumer_key'] #api key 
# consumer_secret = config.params['consumer_secret'] #api secret 
# access_token = config.params['access_token'] #access token  
# access_token_secret = config.params['access_token_secret'] #access secret  
# bear = config.params['bear'] #bearer token

In [3]:
# refer to repo .README for necessary format for keys.py

from keys import *

In [4]:
client = tweepy.Client(bearer_token=bearer_token,\
                       consumer_key = consumer_key,\
                       consumer_secret = consumer_secret,\
                       access_token = access_token,\
                       access_token_secret = access_token_secret)

In [23]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [31]:
# tweets = tweepy.Cursor(api.search_tweets, 'warriors',
#                        lang="en", start_date=game_datetime_start, until=game_datetime_end).items(10)

# print ([tweet for tweet in tweets])

In [11]:
# pulling NBA game specific data

df_data = pd.read_csv("nbaData").drop('Unnamed: 0', axis=1)
df_odds = pd.read_csv("nbaOdds.csv")
df_results = pd.read_csv("nbaResults.csv")

df_data.dateTime = pd.to_datetime(df_data.dateTime)

utc=pytz.UTC
weekPriorDate = datetime.datetime.now() - timedelta(days=6)
weekPriorDate = utc.localize(weekPriorDate) 

df_data.dateTime = df_data.dateTime.apply(lambda x: x.replace(tzinfo=None))

# pulling games that occur within 1 week of today's date

study_games = df_data[df_data.dateTime > weekPriorDate.replace(tzinfo=None)].__id__.unique()

In [95]:
# for tweet in tweepy.Paginator(client.search_recent_tweets, query='Warriors',
#                               tweet_fields=['lang', 'created_at'], max_results=100, start_time=game_datetime_start, end_time=game_datetime_end).flatten(limit=1000):
#     print(tweet.id)

In [96]:
# tweets = client.search_recent_tweets(query=home_team, max_results=100, start_time=game_datetime_start, end_time=game_datetime_end, tweet_fields='lang')

In [135]:
def percentage(part,whole):
    return 100 * float(part)/float(whole)

def clean_tweets(tweet_list):
    tweet_list = [re.sub('RT @\w+: '," ",x) for x in tweet_list]
    tweet_list = [re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x) for x in tweet_list]
    tweet_list = [x.lower() for x in tweet_list]
    return tweet_list

In [138]:
sentiment_results = pd.DataFrame(columns=['gameID', 'homeTeam', 'awayTeam','homePositive', 'homeNegative', 'homeNeutral', 'awayPositive', 'awayNegative', 'awayNeutral', 'total_tweets_analyzed'])
search_limit = 500

for game_id in study_games:
    df = df_data[df_data.__id__ == game_id]
    home_team = df['homeTeam'].values[0]
    away_team = df['awayTeam'].values[0]

    game_datetime = pd.to_datetime(df['dateTime'].values[0])
    game_datetime_end = game_datetime
    game_datetime_start = game_datetime - timedelta(hours=5, minutes=0)
    
    
    print (game_id, home_team, away_team)
    
    # home tweets
    home_tweet_list = []
    home_negative = 0
    home_positive = 0
    home_neutral = 0
    
    for tweet in tweepy.Paginator(client.search_recent_tweets, query=home_team,
                              tweet_fields=['lang', 'created_at'], max_results=100, start_time=game_datetime_start, end_time=game_datetime_end).flatten(limit=search_limit):

        if tweet.lang == 'en':
            home_tweet_list.append(tweet.text)
     
    home_tweet_count = len(set(home_tweet_list))
    
    for tweet in clean_tweets(list(set(home_tweet_list))):
        score = SentimentIntensityAnalyzer().polarity_scores(tweet)
        neg = score['neg']
        neu = score['neu']
        pos = score['pos']

        if neg > pos:
            home_negative += 1
        elif pos > neg:
            home_positive += 1
        elif pos == neg:
            home_neutral += 1
    
    # away tweets
    away_tweet_list = []
    away_negative = 0
    away_positive = 0
    away_neutral = 0
    
    for tweet in tweepy.Paginator(client.search_recent_tweets, query=away_team,
                              tweet_fields=['lang', 'created_at'], max_results=100, start_time=game_datetime_start, end_time=game_datetime_end).flatten(limit=search_limit):

        if tweet.lang == 'en':
            away_tweet_list.append(tweet.text)
    
    away_tweet_count = len(set(away_tweet_list))
    
    for tweet in clean_tweets(list(set(away_tweet_list))):
        score = SentimentIntensityAnalyzer().polarity_scores(tweet)
        neg = score['neg']
        neu = score['neu']
        pos = score['pos']

        if neg > pos:
            away_negative += 1
        elif pos > neg:
            away_positive += 1
        elif pos == neg:
            away_neutral += 1
    
    sentiment_results = sentiment_results.append(pd.DataFrame([[game_id, home_team, away_team, percentage(home_positive, home_tweet_count), percentage(home_negative, home_tweet_count), percentage(home_neutral, home_tweet_count),
                                        percentage(away_positive, away_tweet_count), percentage(away_negative, away_tweet_count), percentage(away_neutral, away_tweet_count), away_tweet_count + home_tweet_count]], 
                                           columns=['gameID', 'homeTeam', 'awayTeam','homePositive', 'homeNegative', 'homeNeutral', 'awayPositive', 'awayNegative', 'awayNeutral', 'total_tweets_analyzed']))


GSDEN4272022 Golden State Warriors Denver Nuggets
MEMMIN4262022 Memphis Grizzlies Minnesota Timberwolves
MIAATL4262022 Miami Heat Atlanta Hawks
MILCHI4272022 Milwaukee Bucks Chicago Bulls
MINMEM4292022 Minnesota Timberwolves Memphis Grizzlies
NOPHO4282022 New Orleans Pelicans Phoenix Suns
PHONO4262022 Phoenix Suns New Orleans Pelicans
TORPHI4282022 Toronto Raptors Philadelphia 76ers
UTADAL4282022 Utah Jazz Dallas Mavericks


In [139]:
sentiment_results

,gameID,homeTeam,awayTeam,homePositive,homeNegative,homeNeutral,awayPositive,awayNegative,awayNeutral,total_tweets_analyzed
0,GSDEN4272022,Golden State Warriors,Denver Nuggets,39.200000,4.800000,56.000000,47.826087,3.804348,48.369565,434
0,MEMMIN4262022,Memphis Grizzlies,Minnesota Timberwolves,49.509804,4.411765,46.078431,52.380952,9.047619,38.571429,414
0,MIAATL4262022,Miami Heat,Atlanta Hawks,34.170854,12.060302,53.768844,36.842105,8.333333,54.824561,427
0,MILCHI4272022,Milwaukee Bucks,Chicago Bulls,48.000000,5.200000,46.800000,49.145299,7.692308,43.162393,484
0,MINMEM4292022,Minnesota Timberwolves,Memphis Grizzlies,43.203883,11.165049,45.631068,45.490196,7.450980,47.058824,461
0,NOPHO4282022,New Orleans Pelicans,Phoenix Suns,44.565217,8.695652,46.739130,48.437500,10.937500,40.625000,376
0,PHONO4262022,Phoenix Suns,New Orleans Pelicans,51.639344,12.295082,36.065574,61.025641,9.230769,29.743590,439
0,TORPHI4282022,Toronto Raptors,Philadelphia 76ers,45.255474,12.043796,42.700730,40.654206,10.280374,49.065421,488
0,UTADAL4282022,Utah Jazz,Dallas Mavericks,42.241379,16.810345,40.948276,46.012270,7.975460,46.012270,395


In [144]:
sentiment_results.to_csv('sentiment_results.csv', index=False)